In [1]:
cd drive/My\ Drive/Colab\ Notebooks/NLP/

/content/drive/My Drive/Colab Notebooks/NLP


In [2]:
ls

file1_similarity.ipynb  model_dir.zip            Untitled0.ipynb
file2_similarity.ipynb  multihead-siamese-nets/
file3_similarity.ipynb  Siamese-LSTM/


In [3]:
# !git clone https://github.com/likejazz/Siamese-LSTM.git

In [4]:
cd Siamese-LSTM/

/content/drive/My Drive/Colab Notebooks/NLP/Siamese-LSTM


In [5]:
ls

data/  predict.py  __pycache__/  README.md  train.py  util.py  word2vec.py


In [6]:
import pandas as pd, gensim
from gensim.models import KeyedVectors
from tensorflow import keras
from util import ManDist, make_w2v_embeddings, split_and_zero_padding

from time import time
import pandas as pd
import numpy as np
import matplotlib

matplotlib.use('Agg')
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

import tensorflow as tf

from tensorflow.python.keras.models import Model, Sequential
from tensorflow.python.keras.layers import Input, Embedding, LSTM, GRU, Conv1D, Conv2D, GlobalMaxPool1D, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from util import make_w2v_embeddings
from util import split_and_zero_padding
from util import ManDist

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [7]:
word2vec = KeyedVectors.load_word2vec_format("./data/GoogleNews-vectors-negative300.bin.gz", binary=True)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:252: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [ ]:
# File paths
TRAIN_CSV = './data/questions.csv'

# Load training set
train_df = pd.read_csv(TRAIN_CSV)
for q in ['question1', 'question2']:
    train_df[q + '_n'] = train_df[q]

# Make word2vec embeddings
embedding_dim = 300
max_seq_length = 20
use_w2v = True

train_df, embeddings = make_w2v_embeddings(train_df, word2vec, 
                                           vector=True, embedding_dim=embedding_dim, 
                                           empty_w2v=not use_w2v)

validation_size = int(len(train_df) * 0.1)
training_size = len(train_df) - validation_size

X = train_df[['question1_n', 'question2_n']]
Y = train_df['is_duplicate']

X_train, X_validation, Y_train, Y_validation = train_test_split(X, 
                                                                Y, test_size=validation_size)

X_train = split_and_zero_padding(X_train, max_seq_length)
X_validation = split_and_zero_padding(X_validation, max_seq_length)

# Convert labels to their numpy representations
Y_train = Y_train.values
Y_validation = Y_validation.values

# Make sure everything is ok
assert X_train['left'].shape == X_train['right'].shape
assert len(X_train['left']) == len(Y_train)

# Model variables
batch_size = 64
n_epoch = 40
n_hidden = 50

# Define the shared model
x = Sequential()
x.add(Embedding(len(embeddings), embedding_dim,
                weights=[embeddings], input_shape=(max_seq_length,), trainable=False))
x.add(tf.keras.layers.Bidirectional(
        tf.keras.layers.LSTM(64, dropout=0.2, return_sequences=True)
    ))
shared_model = x

# The visible layer
left_input = Input(shape=(max_seq_length,), dtype='int32')
right_input = Input(shape=(max_seq_length,), dtype='int32')

# Pack it all up into a Manhattan Distance model
malstm_distance = ManDist()([shared_model(left_input), shared_model(right_input)])
model = Model(inputs=[left_input, right_input], outputs=[malstm_distance])

model.compile(loss='mean_squared_error', optimizer=tf.keras.optimizers.Adam(), metrics=['accuracy'])
model.summary()
shared_model.summary()

early = EarlyStopping(monitor='val_loss',mode='auto', baseline=None, restore_best_weights=False)
# Start trainings
training_start_time = time()
malstm_trained = model.fit([X_train['left'], X_train['right']], Y_train,
                           batch_size=batch_size, epochs=n_epoch,
                           validation_data=([X_validation['left'], X_validation['right']], Y_validation),
                            callbacks=[early])
training_end_time = time()
print("Training time finished.\n%d epochs in %12.2f" % (n_epoch,
                                                        training_end_time - training_start_time))

model.save('./data/SiameseLSTM.h5')

<class 'gensim.models.keyedvectors.Word2VecKeyedVectors'>
Loading word2vec model(it may takes 2-3 mins) ...
10,000 sentences embedded.
20,000 sentences embedded.
30,000 sentences embedded.
40,000 sentences embedded.
50,000 sentences embedded.
60,000 sentences embedded.
70,000 sentences embedded.
80,000 sentences embedded.
90,000 sentences embedded.
100,000 sentences embedded.
110,000 sentences embedded.
120,000 sentences embedded.
130,000 sentences embedded.
140,000 sentences embedded.
150,000 sentences embedded.
160,000 sentences embedded.
170,000 sentences embedded.
180,000 sentences embedded.
190,000 sentences embedded.
200,000 sentences embedded.
210,000 sentences embedded.
220,000 sentences embedded.
230,000 sentences embedded.
240,000 sentences embedded.
250,000 sentences embedded.
260,000 sentences embedded.
270,000 sentences embedded.
280,000 sentences embedded.
290,000 sentences embedded.
300,000 sentences embedded.
310,000 sentences embedded.
320,000 sentences embedded.
330,0

In [8]:
model = keras.models.load_model("./data/SiameseLSTM.h5", custom_objects={'ManDist':ManDist})

In [9]:
sentence1 = "Reports that the NSA eavesdropped on world leaders have \"severely shaken\" relations between Europe and the U.S., German Chancellor Angela Merkel said."
sentence2 = "Germany and France are to seek talks with the US to settle a row over spying, as espionage claims continue to overshadow an EU summit in Brussels."

test_df = pd.DataFrame(data = {'question1': sentence1, 'question2': sentence2, 
                               'question1_n': sentence1, 'question2_n': sentence2}, index =[1])

embedding_dim = 300
max_seq_length = 20
test_df, embeddings = make_w2v_embeddings(test_df, word2vec, vector=True, embedding_dim=embedding_dim, empty_w2v=False)

# Split to dicts and append zero padding.
X_test = split_and_zero_padding(test_df, max_seq_length)

# Make sure everything is ok
assert X_test['left'].shape == X_test['right'].shape

<class 'gensim.models.keyedvectors.Word2VecKeyedVectors'>
Loading word2vec model(it may takes 2-3 mins) ...
Index(['question1', 'question2', 'question1_n', 'question2_n'], dtype='object')


In [10]:
pred = model.predict([X_test['left'], X_test['right']])
print("Sentence 1: {}\n Sentence 2: {}\n Similarity Score: {} ".
      format(sentence1, sentence2, np.average(pred, axis=2)))

Sentence 1: Reports that the NSA eavesdropped on world leaders have "severely shaken" relations between Europe and the U.S., German Chancellor Angela Merkel said.
 Sentence 2: Germany and France are to seek talks with the US to settle a row over spying, as espionage claims continue to overshadow an EU summit in Brussels.
 Similarity Score: [[0.48394203]] 
